# ARC JSON to JSONL Converter (Multi-Test, List Format)

Converts ARC task `.json` files (potentially with multiple test examples) to a `.jsonl` file.
- **train**: String of formatted training examples.
- **test**: List of strings, where each string is a formatted test input.
- **test_answer**: List of raw output grids for all test examples.

Ensures `test[i]` corresponds to the input for `test_answer[i]`.

In [103]:
import json
from pathlib import Path
from tqdm import tqdm
import traceback

In [104]:
# --- Formatting Functions --- 

def format_grid_to_string(grid):
    """Converts a grid (list of lists) to a compact string representation."""
    if not grid or not isinstance(grid, list) or not all(isinstance(row, list) for row in grid):
        return "[]" if isinstance(grid, list) and not grid else ""
    inner_lists = [f"[{''.join(map(str, row))}]" for row in grid]
    return "[" + ",\n".join(inner_lists) + "]"

def get_train_string(task_data, train_template):
    """Formats all training examples into a single string."""
    train_examples = task_data.get("train", [])
    formatted_examples = []
    for idx, example in enumerate(train_examples):
        input_str = format_grid_to_string(example.get("input"))
        output_str = format_grid_to_string(example.get("output"))
        # Keep index for train examples
        formatted_examples.append(train_template.format(index=idx + 1, input=input_str, output=output_str))
    return "\n\n".join(formatted_examples)

def get_test_string_list(task_data, test_template):
    """Formats all test inputs into a LIST of strings."""
    test_examples = task_data.get("test", [])
    formatted_examples_list = []
    for example in test_examples: # No index needed for iteration here
        input_str = format_grid_to_string(example.get("input"))
        # Format using the template (which should NOT contain {index})
        formatted_examples_list.append(test_template.format(input=input_str))
    return formatted_examples_list # Return the list directly

def get_test_answer(task_data):
    """Extracts all test outputs into a list of grids."""
    test_examples = task_data.get("test", [])
    return [example.get("output", []) for example in test_examples]

def format_task_file_to_strings(json_path, train_template, test_template):
    """Loads a single JSON file and returns the formatted train string, test string list, and answer list."""
    try:
        json_path = Path(json_path)
        with open(json_path, 'r', encoding='utf-8') as infile:
            task_data = json.load(infile)
        train_str = get_train_string(task_data, train_template)
        test_str_list = get_test_string_list(task_data, test_template) # Get list of test strings
        test_answer_list = get_test_answer(task_data)
        return train_str, test_str_list, test_answer_list
    except Exception as e:
        print(f"\nError processing file {json_path.name}: {e}")
        traceback.print_exc()
        return None

In [105]:
# --- Test on a Single File --- 

# Define templates
train_template_example = (
    "### Train Example {index}:\n"
    "Input:\n"
    "{input}\n\n"
    "Output:\n"
    "{output}"
)

# Test template WITHOUT index placeholder
test_template_example = (
    "### Test Input:\n" # Consistent header for all test inputs
    "{input}"
)

# Set the sample task filename (!! ADJUST FOLDER/FILENAME AS NEEDED !!)
test_input_folder_example = "../ARC-AGI/data/evaluation"  
example_task_filename = "3e6067c3.json"  # Example file known to have multiple tests

example_task_path = Path(test_input_folder_example) / example_task_filename

print(f"--- Testing custom formatting for: {example_task_path.name} ---")

if example_task_path.is_file():
    result = format_task_file_to_strings(example_task_path, train_template_example, test_template_example)
    if result is not None:
        train_str, test_str_list, test_answer_list = result
        print("\n--- TRAIN ---\n")
        print(train_str)
        
        print("\n--- TEST (List of Input Strings) ---")
        # Print each test input string from the list
        for i, test_input_str in enumerate(test_str_list):
             print(f"--- Test Input {i+1} --- ")
             print(test_input_str)
             print()

        print("\n--- TEST_ANSWER (List of Outputs) ---")
        # Pretty print the list of answers for clarity
        print(json.dumps(test_answer_list, indent=2))
        
        # Verify lengths match
        print(f"\nLength of test input list: {len(test_str_list)}")
        print(f"Length of test answer list: {len(test_answer_list)}")
    else:
        print("An error occurred during formatting.")
else:
    print(f"Error: Example file not found at {example_task_path}. Please check the path.")

print("\n--- End of Single File Test ---")

--- Testing custom formatting for: 3e6067c3.json ---

--- TRAIN ---

### Train Example 1:
Input:
[[88888888888888888888],
[81111881111888111188],
[81221881551888177188],
[81221881551888177188],
[81111881111888111188],
[88888888888888888888],
[88888888888888888888],
[88888888888888888888],
[81111881111888111188],
[81331881991888166188],
[81331881991888166188],
[81111881111888111188],
[88888888888888888888],
[88888881111888111188],
[88888881441888122188],
[88888881441888122188],
[88888881111888111188],
[88888888888888888888],
[82838984828687858888],
[88888888888888888888]]

Output:
[[88888888888888888888],
[81111881111888111188],
[81221881551777177188],
[81221881551777177188],
[81111881111888111188],
[88228888888888866888],
[88228888888888866888],
[88228888888888866888],
[81111881111888111188],
[81331331991888166188],
[81331331991888166188],
[81111881111888111188],
[88888888998888822888],
[88888881111888111188],
[88888881441444122188],
[88888881441444122188],
[88888881111888111188],
[888

In [106]:
# --- Main JSONL Creation Function --- 

def create_formatted_jsonl_from_folder(input_folder, output_jsonl_path, train_template, test_template):
    """Iterates through JSON files and creates the JSONL output with test inputs as a list."""
    input_folder = Path(input_folder)
    output_jsonl_path = Path(output_jsonl_path)
    json_files = sorted(list(input_folder.glob("*.json")))
    num_files = len(json_files)

    if num_files == 0:
        print(f"Warning: No .json files found in {input_folder}. Output file will be empty.")
        output_jsonl_path.touch() 
        return

    print(f"--- Starting JSONL Creation ({num_files} files) ---")
    print(f"Reading from: {input_folder.resolve()}")
    print(f"Writing to:   {output_jsonl_path.resolve()}")

    lines_written = 0
    with open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
        for json_path in tqdm(json_files, desc="Processing"):
            try:
                result = format_task_file_to_strings(json_path, train_template, test_template)
                if result is not None:
                    train_str, test_str_list, test_answer_list = result
                    # Ensure test and answer lists have the same length for consistency
                    if len(test_str_list) == len(test_answer_list):
                        json_obj = {"train": train_str, "test": test_str_list, "test_answer": test_answer_list}
                        outfile.write(json.dumps(json_obj, separators=(',', ':')) + "\n")
                        lines_written += 1
                    else:
                         print(f"\nWarning: Mismatch in length between test inputs ({len(test_str_list)}) and answers ({len(test_answer_list)}) for file {json_path.name}. Skipping.")
                # else: error message already printed by format_task_file_to_strings
            except Exception as e:
                 print(f"\nCritical error processing {json_path.name}: {e}")
                 traceback.print_exc() # Show details for critical errors

    print("--- JSONL Creation Complete ---")
    print(f"Successfully wrote {lines_written} lines to {output_jsonl_path}")
    if lines_written < num_files:
         skipped_count = num_files - lines_written
         print(f"Skipped {skipped_count} file(s) due to errors or length mismatch (see details above)." )

In [107]:
# --- Configuration & Execution --- 

# 1. Define formatting templates
train_template = (
    "### Train Example {index}:\n"
    "Input:\n"
    "{input}\n\n"
    "Output:\n"
    "{output}"
)

# Test template WITHOUT index - applied to each test input individually
test_template = (
    "### Test Input:\n"
    "{input}"
)

# 2. Set Paths (!! CHANGE THESE !!)
input_folder_path = "../ARC-AGI/data/evaluation"  # Folder with your ARC JSON files
output_jsonl_filename = "formatted_arc_test_list.jsonl" # New output file name

# 3. Run the Conversion
input_folder = Path(input_folder_path)
output_file = Path(output_jsonl_filename)

if not input_folder.is_dir():
    print(f"ERROR: Input directory not found: {input_folder.resolve()}")
else:
    create_formatted_jsonl_from_folder(input_folder, output_file, train_template, test_template)

--- Starting JSONL Creation (120 files) ---
Reading from: C:\Users\Lukhausen\github\Lepus\experimental\lukas\ARC-AGI\data\evaluation
Writing to:   C:\Users\Lukhausen\github\Lepus\experimental\lukas\post_train_benchmarking\formatted_arc_test_list.jsonl


Processing: 100%|██████████| 120/120 [00:00<00:00, 442.37it/s]


--- JSONL Creation Complete ---
Successfully wrote 120 lines to formatted_arc_test_list.jsonl


In [108]:
# --- Verification --- 
# Display the first JSON object in the output file.

output_file_verify = Path(output_jsonl_filename) 

if output_file_verify.exists() and output_file_verify.stat().st_size > 0:
    print("\n--- Verification: First line of output file --- ")
    with open(output_file_verify, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        try:
            json_obj = json.loads(first_line)
            print("First JSON object:")
            # Print train part easily
            print("\n'train':")
            print(json_obj.get('train', 'N/A'))
            # Print test list elements
            print("\n'test' (List of strings):")
            test_list = json_obj.get('test', [])
            if isinstance(test_list, list):
                for i, item in enumerate(test_list):
                    print(f"--- Test {i+1} ---")
                    print(item)
            else:
                print(f"Expected list, got: {type(test_list)}")
                print(test_list)
            # Print test_answer list using json.dumps for readability
            print("\n'test_answer' (List of grids):")
            print(json.dumps(json_obj.get('test_answer', []), indent=2))
            # Verify lengths
            print(f"\nLength of 'test' list: {len(json_obj.get('test', []))}")
            print(f"Length of 'test_answer' list: {len(json_obj.get('test_answer', []))}")
        except Exception as e:
            print(f"Error reading/parsing first line: {e}")
            print(f"First line content: {first_line.strip()}")
elif output_file_verify.exists():
     print(f"\n--- Verification --- ")
     print(f"Output file '{output_file_verify}' exists but is empty.")
else:
    print(f"\n--- Verification --- ")
    print(f"Output file '{output_file_verify}' not found.")


--- Verification: First line of output file --- 
First JSON object:

'train':
### Train Example 1:
Input:
[[353366541499439999349941456633],
[533366454199349119439914546633],
[113554669114994554994119664553],
[115345661941914444191491665435],
[699935334399926996299934335399],
[969953333491999669991943333599],
[996911359944699229964499531196],
[999611539154969999694519351169],
[149143995572432442342755993419],
[411934914527344224437254194391],
[991499456455244334425546549941],
[994191444545423443245454441914],
[439999695977557227557795969999],
[349129969577452772547759699219],
[994469997759545555459577988884],
[915496297795464554645977988885],
[915496297795464554645977988885],
[994469997759545555459577988884],
[349129969577452772547759688889],
[439999695977557227557795988889],
[994191444545423443245454488884],
[991499456455244334425546588881],
[411934914527344224437254188881],
[149143995572432442342755993419],
[999611539154969999694519351169],
[996911359944699229964499531196],
[9699533

In [ ]:
# --- Hugging Face Upload (Optional) ---

# Uncomment the following lines to install datasets if needed
# %pip install datasets

try:
    from datasets import Dataset
    print("\n--- Hugging Face Upload --- ")

    output_file_hf = Path(output_jsonl_filename)

    if not output_file_hf.exists() or output_file_hf.stat().st_size == 0:
        print(f"Cannot upload: Output file '{output_file_hf}' not found or is empty.")
    else:
        print(f"Loading dataset from {output_file_hf}...")
        # Loading JSON Lines with lists might need explicit features if auto-detection fails
        # However, from_json usually handles lists of strings and lists of lists correctly.
        dataset = Dataset.from_json(str(output_file_hf))

        # !! CHANGE THIS to your desired Hugging Face repo ID !!
        hf_repo_id = "Lukhausen/arc-agi-lepus-v1-evaluation"

        print(f"Dataset Schema: {dataset.features}") # Check inferred schema
        print(f"Attempting to upload to Hugging Face Hub: {hf_repo_id}")
        print("Ensure you are logged in (`huggingface-cli login`).")
        
        # Upload to Hugging Face Hub (Uncomment to run)
        dataset.push_to_hub(hf_repo_id)
        
        print(f"Upload command for '{hf_repo_id}' is ready.")
        print("Uncomment the `dataset.push_to_hub(...)` line above to execute the upload.")

except ImportError:
    print("\n--- Hugging Face Upload Skipped --- ")
    print("`datasets` library not installed. Cannot upload.")
    print("Install using: %pip install datasets")
except Exception as e:
    print(f"\nAn error occurred during Hugging Face upload preparation: {e}")
    traceback.print_exc()


--- Hugging Face Upload --- 
Loading dataset from formatted_arc_test_list.jsonl...


Generating train split: 0 examples [00:00, ? examples/s]

Dataset Schema: {'train': Value(dtype='string', id=None), 'test': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'test_answer': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None)}
Attempting to upload to Hugging Face Hub: Lukhausen/arc-agi-lepus-v1-evaluation
Ensure you are logged in (`huggingface-cli login`).


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload command for 'Lukhausen/arc-agi-lepus-v1-evaluation' is ready.
Uncomment the `dataset.push_to_hub(...)` line above to execute the upload.


: 